In [80]:
import os
import pandas as pd
import numpy as np
from six.moves import urllib

from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler

#Data initialization





##Data files fetching

In [81]:
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/Snargol/projet-IA-CESI/main/"
DATA_PATH = os.path.join("datasets", "data")

def fetch_data(data_url=DOWNLOAD_ROOT,data_path = DATA_PATH):
  if not os.path.isdir(data_path):
        os.makedirs(data_path)
  employee_url = data_url+"employee_survey_data_full.csv"
  manager_url =  data_url +"manager_survey_data_full.csv"
  general_url =  data_url +"general_data_full.csv"

  employee_path = os.path.join(data_path, "employee_survey_data_full.csv")
  manager_path = os.path.join(data_path, "manager_survey_data_full.csv")
  general_path = os.path.join(data_path, "general_data_full.csv")
  urllib.request.urlretrieve(employee_url, employee_path)
  urllib.request.urlretrieve(manager_url, manager_path)
  urllib.request.urlretrieve(general_url, general_path)

fetch_data()

##Loading datafiles in the NoteBook

In [82]:
def load_data(name):
    csv_path = os.path.join("./datasets/data/"+name+".csv")
    return pd.read_csv(csv_path)

def load_final_data():
  general_data = load_data("general_data_full")
  employee_survey_data = load_data("employee_survey_data_full")
  manager_survey_data = load_data("manager_survey_data_full")
  temp_result = pd.merge(general_data,employee_survey_data,on='EmployeeID')
  result = pd.merge(temp_result,manager_survey_data,on='EmployeeID')
  return result



data = load_final_data()

#Statistical analysis of the dataset

##Categorization of the dataset's columns for statistical analysis

In [83]:
num_columns = ['Age', 'DistanceFromHome', 'Education', 'EmployeeID', 'JobLevel', 'MonthlyIncome', 'NumCompaniesWorked', 'PercentSalaryHike', 'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear', 'YearsAtCompany', 'YearsSinceLastPromotion', 'YearsWithCurrManager', 'EnvironmentSatisfaction', 'JobSatisfaction', 'WorkLifeBalance', 'JobInvolvement', 'PerformanceRating']
num_columns_to_fill = ['NumCompaniesWorked','TotalWorkingYears','EnvironmentSatisfaction','JobSatisfaction','WorkLifeBalance',]
binary_columns = ['Attrition','Gender']
nominal_columns = ['Department','EducationField','JobRole','MaritalStatus']
quali_ordinal_columns = ['BusinessTravel']
quali_ordinal_columns_details =  [
    {
        "label":'BusinessTravel',
        "order":['Non-Travel','Travel_Rarely','Travel_Frequently']
    }
]
quanti_ordinal_columns = ['Education','EnvironmentSatisfaction','WorkLifeBalance','PerformanceRating','JobInvolvement','JobSatisfaction']

columns_to_delete = ['EmployeeCount','StandardHours','Over18']

##Preprocessing of the dataset for statistical analysis

In [84]:
# Deleting the columns with a standard deviation equal to 0 (a unique data for the whole column)
def del_std_of_0(_data):
    _deleted_columns = []
    for each in _data.describe().columns :
      if _data.describe()[str(each)]['std'] == 0:
        _data = _data.drop(columns=[each])
        _deleted_columns.append([each])
    print("Deleted Columns : ")
    print(_deleted_columns)
    return _data
  
data = del_std_of_0(data.copy())


#This function checks if the dataset contains empty cells
def check_contains_nan(_data):
  count_nan = 0
  for each in columns_to_fill:
    temp_count_nan = count_nan + _data[each].isna().sum()
    count_nan= temp_count_nan
  return count_nan


#This function can be used to fill the empty cells with the median value of the column
def fill_data(_data_to_process) :

  imputer = SimpleImputer(strategy="median")
  
  _data_to_process_index = data_num.index
  _data_to_process_labels = data_num.columns


  imputer.fit(_data_to_process)
  data_filled_temp = imputer.transform(_data_to_process)
  data_filled = pd.DataFrame(data_filled_temp,columns=_data_to_process_labels)
  return data_filled

#This function can be used to delete the empty cells of the column 
def delete_nan_row(_data_to_process):
  for each in columns_to_fill:
    _data_to_process.drop(_data_to_process[_data_to_process[each].isna()].index,inplace=True)
  return _data_to_process

data_num = data.select_dtypes(include=[np.number])
data_num = fill_data(data_num)

Deleted Columns : 
[['EmployeeCount'], ['StandardHours']]


In [85]:
print( 'The dataset contains',check_contains_nan(data),'empty cells')
print('* PROCESSING OF THE NUMERICAL COLUMNS *')
data_num = data.select_dtypes(include=[np.number])
data_cat = data.select_dtypes(object)
data_num = fill_data(data_num)
dataset = pd.concat([data_cat,data_num],axis=1)
print( 'The dataset contains',check_contains_nan(dataset),'empty cells')
dataset.head()

The dataset contains 111 empty cells
* PROCESSING OF THE NUMERICAL COLUMNS *
The dataset contains 0 empty cells


,Attrition,BusinessTravel,Department,EducationField,Gender,JobRole,MaritalStatus,Over18,Age,DistanceFromHome,Education,EmployeeID,JobLevel,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating
0,No,Travel_Rarely,Sales,Life Sciences,Female,Healthcare Representative,Married,Y,51.0,6.0,2.0,1.0,1.0,131160.0,1.0,11.0,0.0,1.0,6.0,1.0,0.0,0.0,3.0,4.0,2.0,3.0,3.0
1,Yes,Travel_Frequently,Research & Development,Life Sciences,Female,Research Scientist,Single,Y,31.0,10.0,1.0,2.0,1.0,41890.0,0.0,23.0,1.0,6.0,3.0,5.0,1.0,4.0,3.0,2.0,4.0,2.0,4.0
2,No,Travel_Frequently,Research & Development,Other,Male,Sales Executive,Married,Y,32.0,17.0,4.0,3.0,4.0,193280.0,1.0,15.0,3.0,5.0,2.0,5.0,0.0,3.0,2.0,2.0,1.0,3.0,3.0
3,No,Non-Travel,Research & Development,Life Sciences,Male,Human Resources,Married,Y,38.0,2.0,5.0,4.0,3.0,83210.0,3.0,11.0,3.0,13.0,5.0,8.0,7.0,5.0,4.0,4.0,3.0,2.0,3.0
4,No,Travel_Rarely,Research & Development,Medical,Male,Sales Executive,Single,Y,32.0,10.0,1.0,5.0,1.0,23420.0,4.0,12.0,2.0,9.0,2.0,6.0,0.0,4.0,4.0,1.0,3.0,3.0,3.0


#Preprocessing of the dataset

##Demo of the ordinal variables' transformation

In [86]:
ordinal_encoder = OrdinalEncoder()
business_cat_encoded = ordinal_encoder.fit_transform(data[['BusinessTravel','Attrition']])
business_cat_encoded[:10]

array([[2., 0.],
       [1., 1.],
       [1., 0.],
       [0., 0.],
       [2., 0.],
       [2., 0.],
       [2., 1.],
       [2., 0.],
       [2., 0.],
       [0., 0.]])

##Demo of the categorical variables' transformation

In [87]:
one_hot_encoder = OneHotEncoder()
nom_data = data[['Department','EducationField','JobRole','MaritalStatus']]
nom_encoded = one_hot_encoder.fit_transform(data[['Department','EducationField','JobRole','MaritalStatus','Gender']])
nom_encoded.toarray()

array([[0., 0., 1., ..., 0., 1., 0.],
       [0., 1., 0., ..., 1., 1., 0.],
       [0., 1., 0., ..., 0., 0., 1.],
       ...,
       [0., 1., 0., ..., 0., 0., 1.],
       [0., 0., 1., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 1.]])

##Transformation Pipeline 

In [88]:
num_labels = data.select_dtypes(include=np.number).columns.tolist()
ord_labels = ['BusinessTravel','Attrition']
nom_labels = ['Department','EducationField','JobRole','MaritalStatus','Gender']

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
    ])

full_pipeline = ColumnTransformer(
    [
      ('num',num_pipeline,num_labels),
      ('ord',OrdinalEncoder(),ord_labels),
      ('nom',OneHotEncoder(),nom_labels),
      # ('attrion',LabelBinarizer(),attrition_label),
      # ('gender',LabelBinarizer(),gender_label)
    ]
)


data_labels = pd.get_dummies(data.drop(['BusinessTravel','Over18','Attrition'],axis=1)).columns.tolist()
data_labels.insert(19,'BusinessTravel')
data_labels.insert(20,'Attrition')
raw_data_prepared = full_pipeline.fit_transform(data)
data_prepared = pd.DataFrame(data=raw_data_prepared,columns=data_labels)
data_prepared

,Age,DistanceFromHome,Education,EmployeeID,JobLevel,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating,BusinessTravel,Attrition,Department_Human Resources,Department_Research & Development,Department_Sales,EducationField_Human Resources,EducationField_Life Sciences,EducationField_Marketing,EducationField_Medical,EducationField_Other,EducationField_Technical Degree,Gender_Female,Gender_Male,JobRole_Healthcare Representative,JobRole_Human Resources,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single
0,51.0,6.0,2.0,1.0,1.0,131160.0,1.0,11.0,0.0,1.0,6.0,1.0,0.0,0.0,3.0,4.0,2.0,3.0,3.0,2.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,31.0,10.0,1.0,2.0,1.0,41890.0,0.0,23.0,1.0,6.0,3.0,5.0,1.0,4.0,3.0,2.0,4.0,2.0,4.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,32.0,17.0,4.0,3.0,4.0,193280.0,1.0,15.0,3.0,5.0,2.0,5.0,0.0,3.0,2.0,2.0,1.0,3.0,3.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
3,38.0,2.0,5.0,4.0,3.0,83210.0,3.0,11.0,3.0,13.0,5.0,8.0,7.0,5.0,4.0,4.0,3.0,2.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,32.0,10.0,1.0,5.0,1.0,23420.0,4.0,12.0,2.0,9.0,2.0,6.0,0.0,4.0,4.0,1.0,3.0,3.0,3.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4405,42.0,5.0,4.0,4406.0,1.0,60290.0,3.0,17.0,1.0,10.0,5.0,3.0,0.0,2.0,4.0,1.0,3.0,3.0,3.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4406,29.0,2.0,4.0,4407.0,1.0,26790.0,2.0,15.0,0.0,10.0,2.0,3.0,0.0,2.0,4.0,4.0,3.0,2.0,3.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4407,25.0,25.0,2.0,4408.0,2.0,37020.0,0.0,20.0,0.0,5.0,4.0,4.0,1.0,2.0,1.0,3.0,3.0,3.0,4.0,2.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
4408,42.0,18.0,2.0,4409.0,1.0,23980.0,0.0,14.0,1.0,10.0,2.0,9.0,7.0,8.0,4.0,1.0,3.0,2.0,3.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


##Split of the dataset

In [89]:
attrition_label = data['Attrition']
data_prepared = data_prepared.drop('Attrition',1)



X_train, X_test, y_train, y_test = train_test_split(data_prepared,attrition_label,test_size=0.20,stratify=attrition_label, random_state=42)
X_train, X_cv, y_train, y_cv = train_test_split(X_train,y_train,test_size=0.20,stratify=y_train, random_state=42)

In [90]:
X_train

,Age,DistanceFromHome,Education,EmployeeID,JobLevel,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating,BusinessTravel,Department_Human Resources,Department_Research & Development,Department_Sales,EducationField_Human Resources,EducationField_Life Sciences,EducationField_Marketing,EducationField_Medical,EducationField_Other,EducationField_Technical Degree,Gender_Female,Gender_Male,JobRole_Healthcare Representative,JobRole_Human Resources,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single
313,26.0,2.0,2.0,314.0,1.0,28350.0,3.0,11.0,1.0,5.0,2.0,2.0,2.0,0.0,1.0,3.0,3.0,3.0,3.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3969,42.0,7.0,4.0,3970.0,2.0,57140.0,5.0,11.0,0.0,10.0,2.0,0.0,0.0,0.0,2.0,1.0,2.0,3.0,3.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3786,29.0,23.0,4.0,3787.0,2.0,45370.0,6.0,18.0,1.0,5.0,5.0,1.0,0.0,0.0,3.0,3.0,2.0,4.0,3.0,2.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3679,27.0,2.0,5.0,3680.0,1.0,137700.0,1.0,18.0,1.0,8.0,3.0,8.0,7.0,7.0,4.0,3.0,3.0,4.0,3.0,2.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2751,36.0,7.0,2.0,2752.0,4.0,144110.0,0.0,18.0,3.0,2.0,2.0,1.0,0.0,0.0,4.0,3.0,2.0,4.0,3.0,2.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,46.0,2.0,3.0,176.0,1.0,88370.0,2.0,18.0,0.0,15.0,3.0,3.0,1.0,2.0,3.0,4.0,3.0,2.0,3.0,2.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3515,27.0,6.0,2.0,3516.0,1.0,42600.0,1.0,11.0,2.0,4.0,2.0,4.0,2.0,2.0,4.0,3.0,3.0,3.0,3.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4088,54.0,1.0,3.0,4089.0,2.0,23620.0,6.0,13.0,1.0,9.0,2.0,5.0,1.0,4.0,3.0,1.0,3.0,4.0,3.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
3701,36.0,24.0,3.0,3702.0,3.0,63970.0,4.0,14.0,1.0,11.0,3.0,3.0,0.0,2.0,3.0,1.0,2.0,3.0,3.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [91]:
##Scaling of the dataset

In [92]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
X_test

array([[ 1.7741732 , -1.02230781, -1.86479388, ...,  1.44249299,
        -0.81842684,  0.81842684],
       [ 0.57318132, -0.89890321,  1.0544637 , ...,  1.44249299,
        -0.81842684,  0.81842684],
       [ 1.33744888, -1.02230781,  1.0544637 , ..., -0.69324427,
         1.22185632, -1.22185632],
       ...,
       [-0.84617271,  0.82876112, -0.89170802, ..., -0.69324427,
         1.22185632, -1.22185632],
       [ 0.46400024, -0.77549862,  0.08137784, ...,  1.44249299,
        -0.81842684,  0.81842684],
       [ 0.90072456,  0.58195193,  0.08137784, ..., -0.69324427,
         1.22185632, -1.22185632]])

#Exploitation of the data set